In [ ]:
from fair import FAIR
from fair.interface import fill, initialise
from fair.io import read_properties

import matplotlib.pyplot as pl
import pandas as pd
import pooch
import numpy as np

In [ ]:
climate_response_df = pd.read_csv('../data/fair-calibrations/4xCO2_energy_balance_ebm3.csv')

In [ ]:
f = FAIR()

In [ ]:
scenarios = ['ssp245']
configs = ['UKESM1-0-LL']
species = ['Sulfur', 'Aerosol-radiation interactions', 'Aerosol-cloud interactions']

In [ ]:
f.define_time(1850, 2015, 1)
f.define_scenarios(scenarios)
f.define_configs(configs)
f.define_species(*read_properties(filename='../data/species_configs_properties_vanilla.csv', species=species))

In [ ]:
f.allocate()

In [ ]:
f.fill_from_rcmip()

In [ ]:
fill(f.climate_configs['ocean_heat_capacity'], climate_response_df.loc[0, 'C1':'C3'])
fill(f.climate_configs['ocean_heat_transfer'], climate_response_df.loc[0, 'kappa1':'kappa3'])
fill(f.climate_configs['deep_ocean_efficacy'], climate_response_df.loc[0, 'epsilon'])
fill(f.climate_configs['gamma_autocorrelation'], climate_response_df.loc[0, 'gamma'])

In [ ]:
initialise(f.forcing, 0)
initialise(f.temperature, 0)

In [ ]:
f.fill_species_configs(filename='../data/species_configs_properties_vanilla.csv')

In [ ]:
f.species_configs

In [ ]:
rcmip_emissions_file = pooch.retrieve(
    url="doi:10.5281/zenodo.4589756/rcmip-emissions-annual-means-v5-1-0.csv",
    known_hash="md5:4044106f55ca65b094670e7577eaf9b3",
)
emis_df = pd.read_csv(rcmip_emissions_file)

In [ ]:
bc = emis_df.loc[(emis_df['Scenario']=='historical')&(emis_df['Variable']=='Emissions|BC')&(emis_df['Region']=='World'),'1750':'2014'].values.squeeze()
oc = emis_df.loc[(emis_df['Scenario']=='historical')&(emis_df['Variable']=='Emissions|OC')&(emis_df['Region']=='World'),'1750':'2014'].values.squeeze()
so2 = emis_df.loc[(emis_df['Scenario']=='historical')&(emis_df['Variable']=='Emissions|Sulfur')&(emis_df['Region']=='World'),'1750':'2014'].values.squeeze()

In [ ]:
# 1850 baselines
f.species_configs['baseline_emissions'].loc[dict(specie='Sulfur')] = 4.5444637755469

In [ ]:
# aerosol forcing response - from notebook 01, last cell
f.species_configs['erfari_radiative_efficiency'].loc[dict(specie='Sulfur')] = -4.47634941e-03
f.species_configs['aci_scale'].loc[dict(config='UKESM1-0-LL')] = -1.56516693e+05
f.species_configs['aci_shape'].loc[dict(specie='Sulfur')] = 5.16549484e-08

In [ ]:
f.run()

In [ ]:
# Target -0.49
f.forcing.loc[dict(specie='Aerosol-radiation interactions', config='UKESM1-0-LL', scenario='ssp245')].values[-1]

In [ ]:
# Target -0.88
f.forcing.loc[dict(specie='Aerosol-cloud interactions', config='UKESM1-0-LL', scenario='ssp245')].values[-1]

In [ ]:
# target -1.37
(
    f.forcing.loc[dict(specie='Aerosol-cloud interactions', config='UKESM1-0-LL', scenario='ssp245')].values[-1] +
    f.forcing.loc[dict(specie='Aerosol-radiation interactions', config='UKESM1-0-LL', scenario='ssp245')].values[-1]
)

In [ ]:
pl.plot(
    f.timebounds,
    f.forcing.loc[dict(specie='Aerosol-radiation interactions', config='UKESM1-0-LL', scenario='ssp245')],
    label='ERFari'
)
pl.plot(
    f.timebounds,
    f.forcing.loc[dict(specie='Aerosol-cloud interactions', config='UKESM1-0-LL', scenario='ssp245')],
    label='ERFaci'
)
pl.plot(
    f.timebounds,
    f.forcing.loc[dict(specie='Aerosol-radiation interactions', config='UKESM1-0-LL', scenario='ssp245')] + 
    f.forcing.loc[dict(specie='Aerosol-cloud interactions', config='UKESM1-0-LL', scenario='ssp245')],
    label='Total'
)
pl.legend()